In [1]:
# 구글 코랩에서 제공하는 드라이브 모듈을 임포트함
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 파이선 기본 모듈 중 하나인 os를 임포트함, 이 모듈은 운영체제와 관련한 다양한 기능을 제공함
import os
# 데이터 조작과 분석을 위한 판다스 라이브러리를 임포트함
import pandas as pd

# 폴더 만들기
data_dir = '/content/aiffel/transformer_chatbot/data'
os.makedirs(data_dir, exist_ok=True)

# GitHub에서 직접 읽고 저장
url = 'https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv'
save_path = os.path.join(data_dir, 'ChatbotData.csv')

df = pd.read_csv(url)
# 데이터프레임 객체를 csv객체로 저장함
df.to_csv(save_path, index=False)

# 저장된 파일의 경로 출력
print(f"데이터 저장 완료: {save_path}")

데이터 저장 완료: /content/aiffel/transformer_chatbot/data/ChatbotData.csv


In [3]:
# 저장된 경로에서 다시 불러오기
df_check = pd.read_csv(save_path)
# 내용 확인하기
df_check.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
# 딥러닝 모델을 구축하고 훈련할 텐서플로 프레임워크를 임포트함
import tensorflow as tf
# 다양한 학습용 데이터셋을 쉽게 접근하고 사용할 수 있도록 도와주는 모듈
import tensorflow_datasets as tfds
# 운영체제와 상호작용할 수 있게 해주는 표준 라이브러리
import os
# 정규표현식을 지원함
import re
# 파이선에서 배열객체와 이를 다룰 수 있는 도구를 제공함
import numpy as np
# 파이선에서 데이터를 시각화하는데 가장 많이 사용되는 라이브러리
import matplotlib.pyplot as plt
# 시간관련 기능을 제공하는 모듈
import time
# 판다스는 데이터 조작과 분석을 위한 라이브러리
import pandas as pd

# 위 코드는 데이터 처리, 머신러닝 모델링, 시각화 등 다양한 데이터 과학 작업을
# 수행하기 위한 기본적인 도구들을 포함하고 있음

In [5]:
# 전처리 함수: 간단한 특수문자 제거 + 공백 정리 (숫자 유지)
# 이 함수는 주로 텍스트 데이터를 정제하여 자연어처리(NLP)모델에 적합하도록 만드는 역할을 함
def preprocess_sentence(sentence):
    # 입력된 문장을 문자열로 변환하고 양쪽 끝의 공백을 제거한 후 모든 문자를 소문자로 변환함
    # 이는 대소문자 구분없이 일관된 처리를 하기 위함
    sentence = str(sentence).strip().lower()  # 소문자화 + 양쪽 공백 제거

    # 구두점(punctuation) 주변에 공백 추가
    # 정규표현식을 사용하여 문장 내부의 구두점(물음표,느낌표,쉼표,마침표) 주변에 공백을 추가함
    # 이는 구두점과 인접한 단어와의 구분을 명확히 하여 토큰화 과정에서 유리하게 만듬
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)

    # 여러 공백은 하나로
    # 연속된 공백을 하나의 공백으로 치환함, 이는 데이터의 일관성을 유지하고
    # 이후의 토큰화 과정에서 불필요한 공백으로 인한 문제를 방지함
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 한글, 영어, 숫자, 주요 구두점만 남기고 나머지 제거
    # 정규표현식을 사용하여 한글,영어,숫자,주요 구두점 이외의 모든 문자를 공백으로 치환함
    # 이로써 문장에서 불필요한 기호나 문자를 제거하고 필요한 정보만 남김
    sentence = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9?.!,]+", " ", sentence)

    # 문장의 앞뒤 공백을 한 번 더 제거하여 최종적으로 깔끔한 형태의 문장을 만듬
    sentence = sentence.strip()

    # 전처리된 문장을 반환함
    return sentence

# 위 함수는 주로 텍스트 데이터를 NLP모델에 입력하기 전에 데이터를 정제하는데 사용함. 공백을 정리하고
# 필요없는 문자를 제거함으로써 모델이 더 나은 성능을 낼 수 있도록 도움. 또한 한국어와 영어를
# 모두 처리할 수 있도록 설계됨

In [6]:
# 질문과 대답 각각 전처리 적용
# 이 코드는 데이터프레임에 있는 질문과 답변 열에 대해 전처리함수를 적용하고
# 결과를 확인하는 과정
# 1. 질문과 답변 전처리
# 리스트 컴프리헨션을 사용하여 데이터프레임의 Q열에 있는 모든 질문에 대해
# preprocess_sentence함수를 적용함. 결과는 Questions 리스트에 저장됨.
# 이 리스트는 전처리된 질문들로 구성됨.
questions = [preprocess_sentence(q) for q in df['Q']]
# 마찬가지로, 데이터프레임의 A열에 있는 모든 답변에 대해 preprocess_sentence함수를
# 적용하고 결과를 answers리스트에 저장함
answers = [preprocess_sentence(a) for a in df['A']]

# 확인
# 0부터 4까지 숫자에 대해 반복문을 실행함
for i in range(5):
    # 현재 반복에서의 질문을 출력함.
    print(f"Q: {questions[i]}")
    # 현재 반복에서 답변을 출력함.
    print(f"A: {answers[i]}")
    # 빈 줄을 출력하여 질문-답변 쌍 사이에 공백을 두어 가독성을 높임.
    print()

# 위 코드는 데이터 전처리를 통해 질문과 답변 데이터를 정제하고 그 결과를 간단히 출력하여
# 전처리가 제대로 이루어졌는지 확인함. 전처리된 데이터는 이후의 자연어 처리 작업에 사용될 준비가 됨.
# 이 과정은 데이터의 일관성을 높이고 모델의 성능을 개선하는데 중요한 역할을 함.

Q: 12시 땡 !
A: 하루가 또 가네요 .

Q: 1지망 학교 떨어졌어
A: 위로해 드립니다 .

Q: 3박4일 놀러가고 싶다
A: 여행은 언제나 좋죠 .

Q: 3박4일 정도 놀러가고 싶다
A: 여행은 언제나 좋죠 .

Q: ppl 심하네
A: 눈살이 찌푸려지죠 .



In [7]:
# 모든 문장을 하나로 합쳐 단어장 생성용 코퍼스 만들기
# questions리스트와 answers리스트를 더하여 새로운 리스트 corpus를 생성함.
# 이 corpus리스트는 질문과 답변을 모두 포함하는 단일 텍스트 데이터 집합.
# 이 과정을 통해 전체 데이터셋을 하나의 연속된 텍스트로 통합하여 단어장 생성이나
# 토큰화 등의 작업을 보다 쉽게 수행할 수 있음.
corpus = questions + answers
# 위 코드의 목적은 전처리된 질문과 답변을 하나의 연속된 텍스트로 합쳐서
# 이후의 데이터 처리나 모델 학습에 필요한 코퍼스를 만드는 것임. 이렇게 하면
# 데이터셋 전체를 하나의 단위로 다룰 수 있어, 단어빈도 계산, 토큰화 또는
# 기타 전처리 작업을 일관되게 적용할 수 있음

In [8]:
# SubwordTextEncoder 생성
# 이 코드는 텍스트 데이터를 토큰화하고 이를 기반으로 Subword Text Encoder를 생성하는 과정임.

# SubwordTextEncoder 생성, corpus에 포함된 모든 문장을 기반으로 Subword Text Encoder를 생성함.
# target_vocab_size 매개변수는 토크나이저가 생성할 단어장의 크기를 지정함. 여기서는 2**14로 설정되어
# 있으며 이는 토크나이저가 약 16384개 토큰을 생성한다는 의미임. Subword encoding방식은 자주 등장하지
# 않는 단어를 여러 개의 subword로 나누어 처리하는 방식임.
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    corpus, target_vocab_size=2**14)

# 시작 토큰과 종료 토큰의 ID
# 토큰 ID정의: 현재 최대 단어 수에 해당하는 ID를 할당함.
# 이는 새로운 토큰을 추가하기 위한 공간을 확보하는 것.
START_TOKEN_ID = tokenizer.vocab_size
# START_TOKEN_ID 다음에 위치하는 ID로 종료토큰을 위한 ID임. 이 역시 새로운 토큰을 추가하기 위한 것.
END_TOKEN_ID = tokenizer.vocab_size + 1

# 단어장 크기 (추가 토큰 포함) 갱신
# 전체 단어장의 크기를 갱신함. 원래의 토크나이저 단어 수에 두 개의 추가 토큰(시작토큰과 종료토큰)을
# 더한 값으로 설정함.
VOCAB_SIZE = tokenizer.vocab_size + 2

# 결과 출력 : 최종적으로 생성된 단어장의 크기를 출력함, 이 값은 토크나이저가 생성할 수 있는 총 토큰수를 나타냄
print(f'단어장 크기: {VOCAB_SIZE}')

# 위 코드는 주로 자연어 처리 작업에서 텍스트 데이터를 토큰화하여 모델에 입력할 준비를 하는 과정.
# Subword encoding방식을 사용함으로써 희귀단어도 효과적으로 처리할 수 있으며, 모델의 일반화 능력을
# 높이는데 기여함.

단어장 크기: 21836


In [9]:
# 문장을 토큰화하는 함수
# 입력된 문장을 토큰화하고 특정 길이에 맞게 필터링하는 함수를 정의함.

# 함수정의 : tokenize_and_filter라는 이름의 함수를 정의, 이 함수는 세개의 인자를 받음.
# inputs : 입력문장 리스트, outputs : 출력문장 리스트, max_length : 최대허용 토큰길이
def tokenize_and_filter(inputs, outputs, max_length=40):
    # 토큰화된 입력과 출력을 저장할 빈 리스트를 초기화 함.
    tokenized_inputs, tokenized_outputs = [], []

    # zip을 이용한 문장 쌍 반복 : 입력 문장과 출력 문장을 쌍으로 묶어 반복함. zip함수는 두 리스트를 병렬로 순회할 수 있게 해줌
    for (input_sentence, output_sentence) in zip(inputs, outputs):
        # 각 문장을 정수 시퀀스로 변환, 각 입력 문장을 토큰화하고 시작 토큰 ID와 문장 내부의 각 단어를 토큰화한 후
        # 종료 토큰 ID를 추가함. tokenizer.encode()는 문장을 정수 시퀀스로 변환함.
        input_sentence_tokens = [START_TOKEN_ID] + tokenizer.encode(input_sentence) + [END_TOKEN_ID]
        # 출력 문장도 동일한 방식으로 처리함.
        output_sentence_tokens = [START_TOKEN_ID] + tokenizer.encode(output_sentence) + [END_TOKEN_ID]

        # 최대 길이 체크, 길이 체크 및 필터링. 입력과 출력 문장의 토큰수가 max_length이하인지 확인함.
        # 만약 조건을 만족하지 않으면 해당 문장 쌍은 제외됨. 이는 모델이 처리할 수 있는 최대 길이를 초과하지 않도록
        # 하기 위함.
        if len(input_sentence_tokens) <= max_length and len(output_sentence_tokens) <= max_length:
            # 토큰 리스트에 추가 : 조건을 만족하는 문장 쌍을 각각의 리스트에 추가함.
            tokenized_inputs.append(input_sentence_tokens)
            tokenized_outputs.append(output_sentence_tokens)

    # 패딩 추가 : tf.keras.preprocessing.sequence.pad_sequences 함수를 사용하여 각 문장을 max_length에 맞춰
    # 패딩을 추가함. 'post'옵션은 문장의 끝에 0을 채워 넣는 방식을 의미함.
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=max_length, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=max_length, padding='post')

    # 결과 반환 : 최종적으로 토큰화되고 패딩된 입력과 출력 문장 리스트를 반환함.
    return tokenized_inputs, tokenized_outputs

# 위 함수는 주로 자연어 처리 모델의 입력 데이터를 준비하는데 사용됨. 모든 문장이 일정한 길이로
# 맞춰져 있어야 하므로, 너무 긴 문장은 제외하고 짧은 문장은 패딩을 추가하여 일관된 길이를 유지함.
# 이는 특히 시퀀스 모델(RNN,Transformer)에서 중요함.

In [10]:
# 토큰화 및 필터링 적용
# 앞서 정의한 tokenizer_and_filter함수를 사용하여 질문과 답변 데이터를 토큰화하고
# 필터링하는 과정을 실행함.
# 최대길이 설정: 토큰화할 때 사용할 최대 문장 길이를 40으로 설정함. 이는 모델이 한번에 처리할 수 있는
# 최대 토큰 수를 의미함
MAX_LENGTH = 40
# 토큰화 및 필터링 적용 ; tokenizer_and_filter함수를 호출하여 questions와 answers리스트에 대해 토큰화를 수행함.
# 이 함수는 앞서 정의한대로 문장을 토큰화하고 최대 길이를 초과하는 문장은 제외하며 남은 문장에는 패딩을 추가함.
# 이 함수의 반환값은 questions_tokens와 answers_tokens에 저장됨
questions_tokens, answers_tokens = tokenize_and_filter(questions, answers, MAX_LENGTH)

# 결과출력 : questions_tokens 리스트의 길이를 출력하여 토큰화된 질문 데이터의 개수를 확인함.
print(f'질문 토큰화 결과 크기: {len(questions_tokens)}')
# answers_tokens 리스트의 길이를 출력하여 토큰화된 답변 데이터의 개수를 확인함.
print(f'대답 토큰화 결과 크기: {len(answers_tokens)}')

# 위 코드는 데이터 전처리 과정의 일환으로 모델에 입력할 데이터를 토큰화하고 길이를 조정한 후
# 결과를 확인하는 단계임. 이를 통해 모델이 일관된 길이의 학습 데이터를 학습할 수 있도록 준비함.
# 토큰화된 데이터는 이후의 모델 학습이나 평가 과정에서 사용됨.

질문 토큰화 결과 크기: 11823
대답 토큰화 결과 크기: 11823


In [11]:
# 데이터셋 생성
# 배치 크기 및 버퍼 크기 설정
BATCH_SIZE = 64 # 한 번에 처리할 데이터 샘플의 수를 64로 설정함. 이는 미니 배치 크기를 의미하며, 모델이 한 번의 훈련 단계에서 학습할 데이터 양을 결정
BUFFER_SIZE = 20000 # 데이터셋을 섞기 위한 버퍼의 크기를 20,000으로 설정함. 이 버퍼는 데이터를 무작위로 섞을 때 사용되며, 더 큰 버퍼의 크기는 더 다양한 샘플을 섞는데 도움이 됨.

# 디코더 입력과 타겟 구분 (교사 강요를 위한 준비)
# 데이터셋 생성, 텐서 슬라이스를 기반으로 데이터셋을 생성함.
# 이 데이터셋은 입력 데이터와 타겟 데이터를 포함하며 딕셔너리 구조로 되어 있음.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions_tokens, # 입력 문장들
        'dec_inputs': answers_tokens[:, :-1]  # 디코더 입력으로 사용할 답변 문장들의 마지막 토큰을 제외한 부분, 이는 모델이 다음 단어를 예측할 때 사용할 입력
    },
    {
        'outputs': answers_tokens[:, 1:]  # 타켓 문장들, 답변 문장들의 첫 번째 토큰을 제외한 부분, 이는 모델이 예측해야 할 목표값임.
    }
))

dataset = dataset.cache() # 캐싱, 데이터셋을 캐싱하여 이후의 반복적인 데이터셋 접근 시 속도를 향상시킴. 한번 로드된 데이터를 메모리에 저장해 두었다가 재사용함.
dataset = dataset.shuffle(BUFFER_SIZE) # 섞기, 버퍼 크기만큼 데이터를 무작위로 섞음. 이는 학습 시 데이터의 순서가 편향되지 않도록 하기 위함.
dataset = dataset.batch(BATCH_SIZE) # 배치화, 데이터셋을 배치 크기 64로 나눔. 이는 모델이 한 번에 처리할 데이터의 양을 정의함.
# 미리 로드 ; 데이터셋을 자동으로 최적의 양만큼 미리 로드함, 이는 모델이 데이터를 기다리는 시간을 최소화하여 학습 효율을 높임.
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 데이터셋 출력 ; 최종적으로 준비된 데이터셋의 요약 정보를 출력함.
print(f'데이터셋: {dataset}')

# 위 코드는 주로 딥러닝 모델, 특히 시퀀스-투-시퀀스 모델의 학습 데이터를 준비하는 과정임. 데이터셋을 배치화하고 섞으며, 캐싱하여 모델 학습의 효율성과 성능을 극대화함.

데이터셋: <_PrefetchDataset element_spec=({'inputs': TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(None, 39), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(None, 39), dtype=tf.int32, name=None)})>


In [12]:
# 위치 인코딩을 구현하는 함수들

# get_angles함수는 주어진 위치(pos), 인덱스(i), 그리고 모델의 깊이(d_model)에 따라 각도를 계산함.
def get_angles(pos, i, d_model):
    # 각도를 계산하는 공식, 여기서 10000은 임베딩 차원의 스케일을 조절하는 상수이며 (2*)는 각도의 주파수를 조절하는 부분.
    # 이 공식은 각 위치에 대해 다른 주파수의 사인과 코사인 값을 생성함.
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

# 위치 인코딩 함수 : 주어진 위치의 수와 모델 깊이에 따라 위치 인코딩을 생성함.
def positional_encoding(position, d_model):
    # position은 시퀀스의 길이를 나타내고, d_model은 임베딩 차원임. 이 함수는 각 위치와 각 차원에 대해 get_angles함수를 사용하여
    # 각도를 계산함.
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # 짝수 인덱스에는 사인 함수를 적용함, 이는 위치 인코딩의 짝수 차원에 해당함.
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # 홀수 인덱스에는 코사인 함수를 적용함, 이는 위치 인코딩의 홀수 차원에 해당함.
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    # 결과를 np.newaxis를 사용해 텐서 형태로 변환함, 이는 배치 차원을 추가하여 나중에 모델에 입력할 수 있도록 준비하는 것.
    pos_encoding = angle_rads[np.newaxis, ...]

    # 최종적으로 위치 인코딩을 Tensorflow텐서로 변환하고 데이터 타입을 float32로 캐스팅하여 변환함
    return tf.cast(pos_encoding, dtype=tf.float32)

# 위 코드는 트랜스포머 모델에서 중요한 역할을 하는 위치 인코딩을 구현함. 위치 인코딩은 모델이 입력 시퀀스의
# 순서 정보를 유지할 수 있도록 도와줌. 트랜스포머 모델은 자체적으로 순서 정보를 직접 학습하지 않기 때문에
# 이러한 위치 인코딩을 통해 각 단어의 상대적 위치 정보를 제공함. 이 정보는 특히 시퀀스 모델링 작업에서
# 매우 중요함.

In [13]:
# 어텐션 마스킹 함수
# 패딩 마스크 함수 : 주어진 시퀀스에 대한 패딩 마스크를 생성함. 패딩 마스크는
# 모델이 패딩된 토큰을 무시하도록 돕는 역할을 함
def create_padding_mask(seq):
    # 시퀀스에서 0 값을 갖는 부분을 찾고, 이를 부동 소수점(float32)타입의 불리언 값으로 변환함
    # 0은 패딩 토큰을 나타내므로, 이 마스크는 패딩된 위치를 1로, 그렇지 않은 위치를 0으로 표시함
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # 마스크의 형태를 변경함, 이 형태는 어텐션 메커니즘에서 사용할 수 있도록 준비된 형태
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

# 룩어헤드 마스크 : 주어진 크기(size)에 대한 룩어헤드 마스크를 생성함. 룩어헤드 마스크는
# 디코더에서 미래의 토큰을 보지 못하도록 막아주는 역할을 함
def create_look_ahead_mask(size):
    # size * size크기의 단위행렬을 생성하고, band_part함수는 하삼각 행렬을 추출함.
    # -1,0인자는 하삼각부분을 선택함. 이 행렬을 1에서 빼면 대각선 아래 부분이 1이 되고
    # 나머지가 0인 마스크가 생성됨
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    # 생성된 마스크를 반환함, 이 마스크는 (seq_len,seq_len)형태로, 디코더의 어텐션 매커니즘에서 사용됨
    return mask  # (seq_len, seq_len)

# 위 두 함수는 트랜스포머 모델에서 어텐션 메커니즘을 구현할 때 필수적인 요소임. 패딩 마스크는 입력 시퀀스에서
# 패딩된 부분을 무시하게 하고 룩어헤드 마스크는 디코더가 아직 예측하지 않은 미래 토큰을 보지 못하게 함.
# 이러한 마스크는 모델이 올바른 정보를 참조하도록 유도하며, 학습의 안정성과 성능을 높이는데 기여함.

In [14]:
# 스케일드 닷 프로덕트 어텐션
def scaled_dot_product_attention(q, k, v, mask):
    # 쿼리(Q),키(K),값(V)의 곱셈 : 쿼리(Q)와 키(K)의 행렬 곱셈을 수행함.
    # 여기서 transpose_b=True는 키(K)텐서의 두 번째 축을 전치하여 곱셈을 수행함. 결과적으로
    # (batch_size,seq_len_q,seq_len_k)형태의 텐서가 생성됨
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    # 스케일링 : 키(K) 텐서의 마지막 차원(즉,키의 차원)을 부동 소수점 값으로 변환함. 이는 스케일링을 위한 상수로 사용됨
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    # 쿼리와 키의 곱셈 결과에 스케일링 상수를 나누어 각 원소의 크기를 조정함.
    # 이는 어텐션 로짓(attention logits)이라고 불리며, 소프트맥스 함수를 적용하기 전에 값을 안정화시키는 역할을 함
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # 마스킹 적용 : 만약 마스크가 제공되면 해당 마스크를 어텐션 로짓에 적용함. 마스크 값이 1인 위치에서는
    # 매우 작은 음수값(-1e9)을 더해 해당 위치의 어텐션 가중치가 거의 0이 되도록 함. 이는 패딩된 부분이나 미래의 단어를
    # 무시하는데 사용됨
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # 소프트맥스 적용 : 어텐션 로짓에 소프트맥스 함수를 적용하여 각 위치의 가중치를 계산함, 소프트맥스는 각 위치의 점수를
    # 확률로 변환하며, 이 확률들은 어텐션 매커니즘에서 각 값(Value)의 중요도를 나타냄
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    # 가중치와 값의 곱 : 계산된 어텐션 가중치와 값(Value) 텐서를 곱하여 최종 출력을 생성함. 이 출력은 모델의 다음 레이어로
    # 전달됨
    output = tf.matmul(attention_weights, v)

    # 최종 출력 텐서와 어텐션 가중치를 반환함. 이 두 값은 모델의 학습 과정에서 중요한 역할을 하며 특히 트랜스포머 모델의
    # 여러 레이어에서 재사용됨
    return output, attention_weights

# 위 함수는 트랜스포머 모델의 핵심인 어텐션 매커니즘을 구현하며 입력된 쿼리, 키, 값 텐서를 사용하여 각 단어가 다른 단어들과
# 어떻게 연관되는지를 학습함. 이를 통해 모델은 문맥을 이해하고 더 나은 언어 처리 성능을 발휘할 수 있음.

In [15]:
# 멀티헤드 어텐션 레이어

# 클래스 정의 : 텐서플로의 레이어 클래스를 상속받아 정의됨. 이는 커스텀 레이어를 정의하여
# 케라스 모델에 포함시킬 수 있게 함.
class MultiHeadAttention(tf.keras.layers.Layer):
    # 생성자 : 생성자는 두 개의 매개변수를 받음, d_model(임베딩 차원)과 num_heads(어텐션 헤드의 수)
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        # 임베딩 차원이 헤드 수로 나누어 떨어지는지 확인하는 어설션, 이는 각 헤드가 동일한 차원을 갖도록 하기 위함
        assert d_model % self.num_heads == 0

        # 각 헤드의 깊이를 계산함, 이는 각 헤드가 처리할 수 있는 특성(feature)의 수를 의미함
        self.depth = d_model // self.num_heads

        # 각각 쿼리, 키, 값을 변환하기 위한 밀집층(Dense Layer)임. 이들은 각각의 입력을 새로운 차원으로 매핑하여
        # 어텐션 메커니즘에 적합한 형태로 변환함
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        # 어텐션 출력을 다시 임베딩 차원으로 매핑하기 위한 밀집층
        self.dense = tf.keras.layers.Dense(d_model)

    # 헤드 분할 함수 : 입력 텐서를 여러 헤드로 분할하는 함수
    def split_heads(self, x, batch_size):
        # 입력 텐서를 주어진 배치 크기와 헤드 수에 맞게 재구성함
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        # 텐서의 차원을 재배열하여 각 헤드가 시퀀스 길이와 깊이를 처리할 수 있도록 함
        return tf.transpose(x, perm=[0, 2, 1, 3])

    # 콜백 함수 : 이 함수는 레이어의 실제 연산을 정의함.
    def call(self, v, k, q, mask):
        # 배치 크기를 계산함
        batch_size = tf.shape(q)[0]

        # 쿼리, 키, 값을 각각의 밀집층을 통해 변환함
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        # 변환된 텐서를 여러 헤드로 분할함
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # 스케일드 닷 프로덕트 어텐션 함수를 호출하여 어텐션 매커니즘을 적용함
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        # 어텐션 결과를 원래 형태로 되돌리고 concat_attention으로 결합함
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        # 결합된 어텐션 출력을 다시 한 번 밀집층을 통해 변환하여 최종 출력을 생성함
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        # 최종 출력과 어텐션 가중치를 반환함
        return output, attention_weights

# 위 클래스는 트랜스포머 모델에서 멀티헤드 어텐션 레이어를 구현하는 방법을 보여줌. 여러 개의
# 어텐션 헤드를 사용함으로써 모델은 입력데이터를 다양한 관점에서 동시에 처리할 수 있으며 이는 모델의
# 표현력과 학습 능력을 크게 향상시킴.

In [16]:
# 포지션 와이즈 피드 포워드 네트워크
# 함수 정의 : 이 함수는 두 개의 매개변수를 받음, d_model(임베딩 차원),dff(전달층의 필터 크기 또는 유닛 수)
def point_wise_feed_forward_network(d_model, dff):
    # 시퀀셜 모델 정의 : 텐서플로의 시퀀셜 api를 사용하여 신경망을 정의함. 시퀀셜 모델은 층(레이어)을 순차적으로
    # 쌓을 수 있는 간편한 방법
    return tf.keras.Sequential([
        # 첫 번째 밀집층 : 이 층은 dff 크기의 완전 연결층을 정의하며 활성화 함수로 ReLU를 사용함. 이 층은 입력을
        # 비선형적으로 변환하여 모델이 더 복잡한 패턴을 학습할 수 있도록 함
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        # 두 번째 밀집층 : 이 층은 다시 d_model 크기의 완전 연결층을 정의함. 이 층은 최종적으로 입력 데이터의 차원을
        # 원래의 임베딩 차원(d_model)으로 되돌림
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])
# 이 함수는 트랜스포머 모델 내에서 어텐션 레이어의 출력을 처리하는 역할을 함. 포지션 와이드 피드 포워드 네트워크는
# 어텐션 출력을 비선형 변환하여 더 높은 수준의 표현을 생성하며 이를 통해 모델의 학습 능력을 향상시킴.
# 각 위치의 데이터를 독립적으로 처리하므로, 이 구조는 병렬 처리에 매우 효율적임.

In [17]:
# 인코더 레이어 : 입력 데이터를 처리하고 더 높은 수준의 표현으로 변환하는 역할을 함.
# 클래스 정의 : 텐서플로의 레이어 클래스를 상속받아 정의됨. 이는 커스텀 레이어를 정의하여
# 케라스 모델에 포함시킬 수 있게 함
class EncoderLayer(tf.keras.layers.Layer):
    # 생성자 : 생성자는 네 개의 매개변수를 받음. d_model:임베딩 차원, num_heads:멀티헤드 어텐션에서 사용할 헤드수
    # dff:포지션 와이드 피드 포워드 네트워크의 필터 크기, rate:드롭아웃(dropout)비율(기본값은 0.1),
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        # 멀티헤드어텐션 인스턴스를 생성하여 멀티헤드 어텐션 레이어를 정의함.
        self.mha = MultiHeadAttention(d_model, num_heads)
        # 포지션와이즈 피드 포워드 네트워크를 정의함.
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        # 두 개의 레이어 정규화 레이어를 정의함, 정규화는 각 레이어의 출력을 평균 0, 분산 1로 조정하여
        # 학습을 안정화시키는 역할을 함
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # 두 개의 드랍아웃 레이어를 정의함. 드랍아웃은 과적합을 방지하기 위해 일부 뉴런을 무작위로
        # 비활성화하는 기법
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    # 콜백함수
    # 이 함수는 인코더 레이어의 실제 연산을 정의함.
    def call(self, x, training, mask):
        # 멀티헤드 어텐션 레이어를 호출하여 입력 x에 대해 어텐션을 수행함. 여기서 v,k,q 모두 x로 설정되어
        # 자기 어텐션(self-attention)을 수행함. mask는 패딩된 부분을 무시하기 위해 사용됨.
        attn_output, _ = self.mha(v=x, k=x, q=x, mask=mask)
        # 어텐션 출력에 드랍아웃을 적용함
        attn_output = self.dropout1(attn_output, training=training)
        # 입력x와 어텐션 출력을 더한 후, 레이어 정규화를 적용함. 이는 잔차 연결(residual connection)을 구현하는 방법임
        out1 = self.layernorm1(x + attn_output)

        # 포지션 와이즈 피드 포워드 네트워크를 통과시켜 out1을 처리함.
        ffn_output = self.ffn(out1)
        # 포워드 네트워크 출력에 드랍아웃을 적용함.
        ffn_output = self.dropout2(ffn_output, training=training)
        # 이전의 out1과 ffn_output을 더한 후 다시 레이어 정규화를 적용함.
        out2 = self.layernorm2(out1 + ffn_output)

        # 최종 출력을 반환함
        return out2

# 위 클래스는 트랜스포머 모델의 인코더 레이어를 구현하며, 입력 데이터를 처리하고, 어텐션 및 피드 포워드 네트워크를 통해
# 더 높은 수준의 표현으로 변환함. 이 구조는 병렬 처리가 용이하며 모델의 깊이와 복잡성을 증가시키면서도 학습을 안정화시키는
# 특징이 있음

In [18]:
# 디코더 레이어 정의
# 디코더 레이어는 인코더의 출력을 기반으로 입력 시퀀스를 정의하고 이를 통해 예측 단어를
# 생성하는 역할을 함.
# 클래스 정의 : 텐서플로의 레이어 클래스를 상속받아 정의됨, 이는 커스텀 레이어를 정의하여
# 케라스 모델에 포함시킬 수 있게 함
class DecoderLayer(tf.keras.layers.Layer):
    # 생성자 : 네 개의 매개변수를 받음, d_model: 임베딩 차원, num_heads: 멀티헤드 어텐션에서 사용할 헤드수
    # dff: 포지션 와이즈 피드 포워드 네트워크의 필터 크기, rate: 드랍아웃(dropout) 비율(기본값은 0.1)
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        # 두 개의 멀티헤드 어텐션 인스턴스를 생성하여 멀티헤드 어텐션 레이어를 정의함
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        # point_wise_feed_forward_network 함수를 호출하여 포지션 와이즈 피드 포워드 네트워크를 정의함
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        # 세 개의 레이어 정규화 레이어를 정의함, 레이어 정규화는 각 레이어의 출력을 평균 0, 분산 1로 조정하여
        # 학습을 안정화시키는 역할을 함
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # 세 개의 드롭아웃 레이어를 정의함, 드롭아웃은 과적합을 방지하기 위해 일부 뉴런을 무작위로
        # 비활성화하는 기법
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    # 콜백함수 : 디코더 레이어의 실제 연산을 정의함.
    def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        # 첫 번째 멀티헤드 어텐션 레이어를 호출하여 입력 x에 대해 자기 어텐션을 수행함.
        # 이때 look_ahead_mask는 미래의 단어를 보지 못하도록 하는 마스크임.
        attn1, attn_weights_block1 = self.mha1(v=x, k=x, q=x, mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        # 첫 번째 어텐션 출력에 드랍아웃과 잔차 연결을 적용한 후, 레이어 정규화를 수행함.
        out1 = self.layernorm1(attn1 + x)

        # 두 번째 멀티헤드 어텐션 레이어를 호출하여 인코더 출력(enc_output)과 첫 번째
        # 어텐션 출력(out1)을 사용함. 이때 padding_mask는 패딩된 부분을 무시하는 마스크임.
        attn2, attn_weights_block2 = self.mha2(
            v=enc_output, k=enc_output, q=out1, mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        # 두 번째 어텐션 출력에 드랍아웃과 잔차 연결을 적용한 후, 다시 레이어 정규화를 수행함.
        out2 = self.layernorm2(attn2 + out1)

        # 포지션 와이즈 피드 포워드 네트워크를 통과시켜 out2를 처리함
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        # 피드 포워드 네트워크 출력에 드랍아웃과 잔차 연결을 적용한 후, 마지막으로 레이어 정규화를 수행함.
        out3 = self.layernorm3(ffn_output + out2)

        # 최종 출력과 두 개의 어텐션 가중치를 반환함
        return out3, attn_weights_block1, attn_weights_block2

# 이 클래스는 트랜스포머 모델의 디코더 레이어를 구현하며, 인코더의 출력과 함께 입력 시퀀스를 처리하여
# 예측 단어를 생성하는 역할을 함, 두 번의 멀티헤드 어텐션 레이어를 통해 모델은 이전 단어의 정보와 인코더의
# 컨텍스트 정보를 모두 활용할 수 있음. 이 구조는 병렬 처리가 용이하며, 모델의 깊이와 복잡성을 증가시키면서도
# 학습을 안정화시키는 특징이 있음.

In [19]:
# 인코더 정의 : 입력 시퀀스를 받아 이를 더 높은 수준의 표현으로 변환하는 역할을 함
# 클래스 정의 : 텐서플로의 레이어 클래스를 상속받아 정의됨, 이는 커스텀 레이어를 정의하여
# 케라스 모델에 포함시킬 수 있게 함.
class Encoder(tf.keras.layers.Layer):
    # 생성자 : 일곱개의 매개변수를 받음, num_layers : 인코더에 포함될 레이어 수, d_model : 임베딩 차원,
    # num_heads : 멀티헤드 어텐션에서 사용할 헤드의 수, dff : 포지션 와이즈 피드 포워드 네트워크의 필터 크기
    # input_vocab_size : 입력 단어 사전의 크기, maximum_position_encoding : 최대 위치 인코딩 길이
    # rate : 드랍아웃 비율(초기0.1)
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        # 입력 단어를 d_model 차원의 벡터로 변환하는 임베딩 레이어
        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        # 위치 인코딩을 생성하는 객체로 시퀀스의 각 위치에 대한 정보를 추가함.
        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                              self.d_model)

        # 객체의 리스트로, num_layers만큼 반복하여 생성됨
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]

        # 드랍아웃 레이어임
        self.dropout = tf.keras.layers.Dropout(rate)

    # 콜백함수 : 이 함수는 인코더의 실제 연산을 정의함.
    def call(self, x, training, mask):
        # 입력 시퀀스의 길이를 계산함
        seq_len = tf.shape(x)[1]

        # 입력 시퀀스를 임베딩하여 d_model 차원의 벡터로 변환함
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        # 임베딩 값을 스케일링 함, 이는 임베딩 차원에 제곱근을 취한 값으로 곱하는 것으로,
        # 작은 그래이디언트 문제를 완화하는 역할을 함
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # 위치 인코딩을 추가함, 위치 인코딩은 시퀀스의 각 위치에 대한 정보를 제공하여
        # 모델이 순서 정보를 학습할 수 있게 함
        x += self.pos_encoding[:, :seq_len, :]
        # 드랍아웃을 적용함
        x = self.dropout(x, training=training)
        # 인코더 레이어를 순차적으로 적용함, 각 레이어는 이전 레이어의 출력을 입력으로 받아 처리함.
        for i in range(self.num_layers):
            x = self.enc_layers[i](x=x, training=training, mask=mask)
        # 최종적으로 인코더의 출력을 반환함, 이 출력은 인코더 입력 시퀀스의 더 높은 수준의 표현임.
        return x  # (batch_size, input_seq_len, d_model)

# 이 클래스는 트랜스포머 모델의 인코더 부분을 구현하며, 입력 시퀀스를 처리하고 위치 인코딩과 여러 개의
# 인코더 레이어를 통해 더 높은 수준의 표현으로 변환함. 이 구조는 병렬 처리가 용이하며, 모델의 깊이와
# 복잡성을 증가시키면서도 학습을 안정화시키는 특징이 있음.

In [20]:
# 디코더 : 인코더의 출력과 함께 입력 시퀀스를 처리하여 예측 단어를 생성하는 역할을 함
# 정의 : 텐서플로의 레이어 클래스를 상속받아 정의됨, 이는 커스텀 레이어를 정의하여 케라스 모델에
# 포함시킬 수 있게 함.
class Decoder(tf.keras.layers.Layer):
    # 생성자는 일곱 개의 매개변수를 받음, num_layers : 디코더에 포함될 레이어 수, d_model : 임베딩 차원
    # num_heads : 멀티헤드 어텐션에서 사용할 헤드의 수, dff: 포지션 와이즈 피드 포워드 네트워크의 필터크기
    # target_vocab_size : 타겟 단어 사전의 크기, maximum_position_encoding : 최대 위치 인코딩 길이
    # rate : 드랍아웃 비율(기본값은 0.1)
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        # 타겟 단어를 d_model 차원의 벡터로 변환하는 임베딩 레이어
        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        # 위치 인코딩을 생성하는 객체로 시퀀스의 각 위치에 대한 정보를 추가함
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        # 객체의 리스트로, num_layers만큼 반복하여 생성됨
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]
        # 드랍아웃 레이어임.
        self.dropout = tf.keras.layers.Dropout(rate)

    # 콜백함수 : 이 함수는 디코더의 실제 연산을 정의함
    def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
        # 입력 시퀀스의 길이를 계산함
        seq_len = tf.shape(x)[1]
        # 어텐션 가중치를 저장할 딕셔너리를 초기화함
        attention_weights = {}
        # 입력 시퀀스를 임베딩하여 d_model차원의 벡터로 변환함
        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        # 임베딩값을 스케일링함, 이는 임베딩의 차원에 제곱근을 취한 값으로 곱하는 것으로,
        # 작은 그래이디언트 문제를 완화하는 역할을 함
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # 위치 인코딩을 추가함, 위치 인코딩은 시퀀스의 각 위치에 대한 정보를 제공하여
        # 모델이 순서 정보를 학습할 수 있게 함
        x += self.pos_encoding[:, :seq_len, :]
        # 드랍아웃을 적용함.
        x = self.dropout(x, training=training)
        # 디코더 레이어를 순차적으로 적용함, 각 레이어는 이전 레이어의 출력과 인코더 출력을
        # 입력으로 받아 처리함.
        for i in range(self.num_layers):
            # 각 디코더 레이어의 출력과 두 개의 어텐션 가중치를 계산함
            x, block1, block2 = self.dec_layers[i](
                x=x,
                enc_output=enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask
            )
            # 어텐션 가중치를 딕셔너리에 저장함.
            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2
        # 최종적으로 디코더의 출력과 어텐션 가중치를 반환함
        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

# 위 클래스는 트랜스포머 모델의 디코더 부분을 구현하며, 인코더의 출력과 함께 입력 시퀀스를 처리하여
# 예측 단어를 생성하는 역할을 함, 두 번의 멀티헤드 어텐션 레이어를 통해 모델은 이전 단어의 정보와 인코더의
# 컨텍스트 정보를 모두 활용할 수 있음. 이 구조는 병렬 처리가 용이하며, 모델의 깊이와 복잡성을 증가시키면서도
# 학습을 안정화시키는 특징이 있음

In [21]:
# 트랜스포머 모델에서 사용되는 다양한 종류의 마스크를 생성하는 함수들 정의
# 이 마스크들은 어텐션 매커니즘에서 중요한 역할을 하며, 모델이 올바른 정보를 참조하도록 도움
# 패딩 마스크 생성: 입력 시퀀스에서 패딩 토큰(0)에 해당하는 위치를 마스킹
# 패딩 마스크 생성함수 ; 이 함수는 입력 시퀀스에서 패딩 토큰(0)에 해당하는 위치를
# 마스킹하는 역할을 함
def create_padding_mask(seq):
    # 입력 시퀀스에서 0값을 갖는 부분을 찾아 부동 소수점 값으로 변환함, 0은 패딩된 토큰을 나타냄
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # 마스크의 형태를 (배치 크기,1,1,시퀀스 길이)로 변환함, 이 형태는 어텐션 메커니즘에서 사용될 수 있도록
    # 준비된 형태
    return seq[:, tf.newaxis, tf.newaxis, :]  # (배치 크기, 1, 1, 시퀀스 길이)

# 미래 토큰 마스킹 (Look-ahead mask): 디코더가 미래의 토큰을 보지 못하게 막음
# 미래 토큰 마스킹 함수 : 디코더가 미래의 토큰을 보지 못하게 막는 룩어헤드 마스크를 생성함.
def create_look_ahead_mask(size):
    # (size,size)크기의 단위행렬을 생성하고 하삼각(triangular part)을 추출하여 1에서 뺌, 이렇게 하면
    # 대각선 아래의 원소들이 1이 되고, 나머지 원소들은 0이 됨. 이는 디코더가 아직 예측하지 않은
    # 미래의 단어를 보지 못하도록 함
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    # 생성된 마스크를 반환함, 이 마스크는 (시퀀스 길이,시퀀스 길이)형태임
    return mask  # (시퀀스 길이, 시퀀스 길이)

# 전체 마스크 생성 함수: 인코더/디코더 패딩 마스크와 Look-ahead 마스크 결합
# 전체 마스크 생성 함수 : 인코더와 디코더 입력에 필요한 모든 마스크를 생성함.
def create_masks(inp, tar):
    # 인코더 입력에 대한 패딩 마스크
    # 인코더 입력에 대한 패딩 마스크 생성
    enc_padding_mask = create_padding_mask(inp)

    # 디코더 입력에 대한 look-ahead 마스크와 패딩 마스크 결합
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    # 디코더 입력에 대한 룩어헤드 마스크와 패딩 마스크를 결합함, tf.maximum함수를 사용하여
    # 두 마스크 중 더 큰 값을 선택함, 이는 디코더가 패딩된 부분과 미래의 단어를 모두 무시하도록 함
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    # 디코더의 인코더 출력에 적용할 패딩 마스크
    # 디코더의 인코더 출력에 적용할 패딩 마스크를 생성함.
    dec_padding_mask = create_padding_mask(inp)
    # 반환된 세 가지 마스크는 모델의 어텐션 메커니즘에서 사용되어, 모델이 올바른 정보를 참조하고
    # 학습할 수 있도록 도움.
    return enc_padding_mask, combined_mask, dec_padding_mask

# 위 코드는 트랜스포머 모델의 어텐션 메커니즘에서 중요한 역할을 하는 마스크를 생성하는 함수들을 정의함.
# 패딩 마스크는 입력 시퀀스에서 패딩된 부분을 무시하게 하고 룩어헤드 마스크는 디코더가 미래의 단어를
# 참조하지 못하게 함, 이 두 마스크를 결합하여 디코더가 올바르게 학습할 수 있도록 함. 이러한 마스크는
# 모델의 학습 안정성과 성능을 높이는데 기여함.

In [22]:
# 트랜스포머 모델 : 이 클래스는 인코더,디코더, 그리고 최종 출력 레이어를 포함하여 입력 시퀀스를 처리하고
# 예측 단어를 생성하는 역할을 함,
# 클래스 정의 : 텐서플로의 tf.keras.Model을 상속받아 정의됨, 이는 커스텀 케라스 모델을 정의하여
# 쉽게 사용할 수 있게 함.
class Transformer(tf.keras.Model):
    # 생성자는 일곱 개의 매개변수를 받음, num_layers : 인코더와 디코더에 포함될 레이어의 수, d_model : 임베딩 차원
    # num_heads : 멀티헤드 어텐션에서 사용할 헤드의 수, dff: 포지션 와이즈 피드 포워드 네트워크의 필터 크기, input_vocab_size : 입력 단어 사전의 크기
    # target_vocab_size : 타겟 단어 사전의 크기, pe_input 및 pe_target : 인코더와 디코더의 최대 위치 인코딩 길이, rate:드랍아웃 비율(기본값을 0.1)
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        # 인코더 객체를 초기화함.
        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)
        # 디코더 객체를 초기화함.
        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                             target_vocab_size, pe_target, rate)
        # 최종 출력 레이어로 디코더의 출력을 타겟 단어 사전 크기로 매핑하는 밀집층
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    # 콜백함수 : 이 함수는 모델의 실제 연산을 정의함,inputs : 모델에 입력되는 데이터로 딕셔너리 형태로 주어지며 'inputs'와
    # 'dec_inputs'를 포함함.
    def call(self, inputs, training=False):
        # Keras 모델은 inputs를 리스트나 딕셔너리로 받음
        inp, tar = inputs['inputs'], inputs['dec_inputs']
        # (...) 앞서 정의한 마스크 생성 함수를 호출하여 필요한 마스크들을 생성함.
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
        # 인코더를 호출하여 입력 시퀀스를 처리하고 인코더 출력을 얻음
        enc_output = self.encoder(
            x=inp,
            training=training,
            mask=enc_padding_mask
        )
        # 디코더를 호출하여 인코더 출력과 디코더 입력을 처리하고 디코더 출력과 어텐션 가중치를 얻음.
        dec_output, attention_weights = self.decoder(
            x=tar,
            enc_output=enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )
        # 최종 출력 레이어를 통해 디코더 출력을 타겟 단어 사전 크기로 매핑함
        final_output = self.final_layer(dec_output)
        # 최종 출력을 반환함
        return final_output

# 위 클래스는 트랜스포머 모델을 구현하며 입력 시퀀스를 처리하고 예측 단어를 생성하는 역할을 함
# 인코더와 디코더를 통해 입력 데이터를 더 높은 수준의 표현으로 변환하고 최종 출력 레이어를 통해
# 예측 단어를 생성함, 이 구조는 병렬 처리가 용이하며 모델의 깊이와 복잡성을 증가시키면서도 학습을
# 안정화시키는 특징이 있음

In [23]:
# 트랜스포머 모델을 정의하고 초기화하는 과정을 설명함
# 학습 파라미터 설정
num_layers = 4 # 트랜스포머 모델의 층수, 여기서는 4개의 층
d_model = 128 # 모델의 임베딩 차원. 즉 입력 데이터가 변화되는 벡터의 차원이 128
dff = 512 # 피드포워드 네트워크의 뉴런 수, 여기서는 512개의 뉴런을 사용함, 이는 각 멀티헤드 어텐션 레이어 뒤에 위치하는 완전 연결 레이어의 크기
num_heads = 8 # 멀티헤드 어텐션의 개수, 여기서는 8개의 멀티헤드를 사용함. 이는 어텐션 메커니즘이 병렬로 여러 부분에서 작동함
dropout_rate = 0.1 # 드랍아웃 기법을 사용할 때 비율, 0.1은 전체 뉴런 중 10%가 랜덤하게 드랍아웃됨, 과적합을 방지하기 위해 사용됨

# 모델 인스턴스 생성
# 트랜스포머 클래스의 인스턴스를 생성함. 이 인스턴스는 위에서 정의한 하이퍼파라미터를 사용하여 초기화됨
transformer = Transformer(
    num_layers=num_layers, # 앞에서 정의한 하이퍼 파라미터들
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=VOCAB_SIZE, # 입력 단어 사전의 크기
    target_vocab_size=VOCAB_SIZE, # 출력 단어 사전의 크기
    pe_input=MAX_LENGTH, # 위치 인코딩(Positional Encoding)의 최대 길이
    pe_target=MAX_LENGTH, # 상동
    rate=dropout_rate # 드랍아웃의 비율
)

# 위 코드는 트랜스포머 모델의 구조를 정의하고 필요한 모든 하이퍼파라미터를 설정하여 모델을 초기화하는 과정
# 이를 통해 트랜스포머 모델을 학습시킬 준비가 완료됨

In [24]:
# 손실 함수 정의 : 이 부분은 실제 레이블(real)에서 패딩(padding)부분을 제외하기 위한 마스크를 생성함
def loss_function(real, pred):
    # equal()는 각 요소가 0인지 여부를 논리값으로 반환하고 logical_not은 그 논리를 반전시킴
    # 결과적으로 패딩이 아닌 실제 데이터 포인트에만 True값을 갖는 마스크가 생성됨
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # sparse_categorical_crossentropy함수는 다중 클래스 분류 문제에서 사용되는 손실함수임
    # real은 실제 레이블이고 pred는 모델의 예측값임. =True는 모델의 출력이 로짓 형태임을 나타냄
    loss_ = tf.keras.losses.sparse_categorical_crossentropy(
        real, pred, from_logits=True)
    # 앞서 만든 마스크를 손실 텐서와 같은 데이터 타입으로 변환함, 이는 연산을 가능하게 함
    mask = tf.cast(mask, dtype=loss_.dtype)
    # 손실 텐서에 마스크를 곱하여 패딩 부분을 손실 계산에서 제외함
    loss_ *= mask
    # 패딩을 제외한 유효한 손실의 합을 전체 마스크의 합으로 나누어 평균 손실을 반환함
    # 이는 패딩이 포함된 데이터셋에서도 정확도 손실을 계산할 수 있게 함
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

# 정확도 계산 함수 정의
def accuracy_function(real, pred):
    # tf.argmax()는 예측값(pred)의 각 위치에서 가장 높은 값을 가지는 인덱스를 찾음
    # 이 인덱스는 예측된 레이블을 나타냄, real과 비교하여 예측이 맞으면 T,틀리면 F가 됨
    accuracies = tf.equal(real, tf.cast(tf.argmax(pred, axis=2), dtype=tf.int32))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # logical_and()는 앞서 만든 마스크와 예측 정확도를 논리곱으로 결합하여 패딩 부분을 제외하고
    # 실제 데이터 포인트에서의 정확도만 남김
    accuracies = tf.math.logical_and(mask, accuracies)
    # tf.cast()는 논리값(T/F)을 부동소수점(1.0/0.0)으로 변환함
    accuracies = tf.cast(accuracies, dtype=tf.float32)
    # 마스크를 부동소수점 값으로 변환함
    mask = tf.cast(mask, dtype=tf.float32)
    # 유효한 데이터 포인트에서의 정확도의 합을 전체 마스크의 합으로 나누어 평균 정확도를 반환함
    # 이는 패딩이 포함된 데이터셋에서도 정확한 정확도를 계산할 수 있게 함
    return tf.reduce_sum(accuracies) / tf.reduce_sum(mask)

# 이 두 함수는 트랜스포머 모델의 성능을 평가하기 위해 손실과 정확도를 계산하는데 사용하며 패딩 토큰을
# 고려하여 실제 데이터 포인트만을 대상으로 계산하도록 설계됨

In [25]:
# 학습률 스케줄러 정의
# 텐서플로의 LearningRateSchedule를 상속받아 사용자 정의 학습률 스케줄을 구현함
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    # 생성자: d_model: 모델의 임베딩 차원, warmup_steps: 학습률이 점진적으로 증가하는 웜업 단계의 길이
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        # 캐스팅을 통해 float32 타입으로 변환됨
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    # 이 메서드는 주어진 스텝(step)에 대한 학습률을 계산함
    def __call__(self, step):
        # step은 훈련 스텝 수를 나타내며, float32타입으로 명시적으로 변환됨
        step = tf.cast(step, tf.float32)
        # 아래는 학습률 스케줄을 정의하는 두 가지 주요 요소
        arg1 = tf.math.rsqrt(step) # 스텝의 제곱근 역수를 계산함
        arg2 = step * (self.warmup_steps ** -1.5) # 웜업 단계 이후에는 스텝에 비례하여 감소하는 항을 추가함
        # 최종 학습률은 rsqrt(self.d_model)에 minimum()를 곱하여 계산됨, 이는 모델의 차원에 따라 조정되고
        # 웜업 단계 동안 점진적으로 증가하다가 이후에는 감소하는 스케줄을 따름
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# 학습률 설정 및 옵티마이저 정의
# 학습률 설정하기
learning_rate = CustomSchedule(d_model)
# 옵티마이저 정의 : Adam을 사용하여 옵티마이저를 정의함, 여기서 learning_rate는 앞서 정의한 커스텀 스케줄을 사용함
optimizer = tf.keras.optimizers.Adam(
    # beta_1, beta_2, epsilon은 Adam 옵티마이저의 하이퍼파라미터
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# 체크포인트 설정
# 체크포인트 디렉토리
checkpoint_path = './checkpoints/transformer'
# 체크포인트 객체 생성
ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# 위 코드는 트랜스포머 모델의 학습률을 동적으로 조절하고 학습 중간에 모델과 옵티마이저의 상태를 저장하여
# 나중에 재개하거나 모델을 저장할 수 있도록 설정함

In [26]:
# 학습 함수 정의
# 트랜스포머 모델의 한 번의 학습 반복(스텝)을 정의하는 함수, 이 함수는 텐서플로의 tf.function데코레이터를
# 사용하여 그래프로 컴파일되어 효율적인 실행을 보장함.
@tf.function
# 학습 함수 정의
def train_step(inp, tar):
    # 입력 및 타겟 데이터 준비
    # 타겟 데이터에서 마지막 요소를 제외한 부분을 입력으로 사용함, 이는 모델이 다음 단어를 예측하는 방식
    tar_inp = tar[:, :-1]
    # 타겟 데이터의 첫 번째부터 마지막 요소까지를 실제 레이블로 사용함
    tar_real = tar[:, 1:]
    # 그래이디언트 컨텍스트 관리자 사용, 그래이디언트 테입을 사용하여 자동 미분을 수행함
    # 이 컨텍스트 내에서 수행되는 모든 연산은 그래프로 기록됨
    with tf.GradientTape() as tape:
        # 모델 예측 수행 : 트랜스포머 모델에 입력과 타켓입력을 전달하여 예측값을 얻음.
        predictions = transformer(
            {
                'inputs': inp,
                'dec_inputs': tar_inp
            },
            # 아래 코드는 모델이 학습모드임을 나타냄
            training=True
        )
        # 손실계산 : 정의된 손실함수를 사용하여 실제 레이블과 모델의 예측값간의 손실을 계산함
        loss = loss_function(tar_real, predictions)
    # 그래이디언트 계산 및 적용
    # 그래이디언트 테입을 사용하여 손실에 대한 모델의 가중치들의 그래이디언트를 계산함
    gradients = tape.gradient(loss, transformer.trainable_variables)
    # 계산된 그래이디언트를 옵티마이저에 적용하여 모델의 가중치를 업데이트함
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    # 메트릭 업데이트
    # 현재 스텝의 손실을 기록함
    train_loss(loss)
    # 현재 스텝의 정확도를 기록함
    train_accuracy(accuracy_function(tar_real, predictions))

# 위 함수는 트랜스포머 모델의 한 번 학습스텝을 정의하며, 모델의 예측을 생성하고 손실과 정확도를
# 계산한 후 그래이디언트를 통해 모델을 업데이트함. tf.function데코레이터는 이 함수를 텐서플로
# 그래프로 변환하여 최적화된 환경에서 실행되도록 함

In [27]:
# tqdm 라이브러리 설치 및 가져오기
# 주피터 노트북 환경에서 tqdm 라이브러리를 설치함. 이 라이브러리는 학습 중
# 진행 상황을 시각적으로 보여주는 프로그레스 바를 제공함
!pip install tqdm
# tqdm 모듈 가져오기, 주피터 노트북에서 사용할 수 있는 tqdm의 노트북 버전을 가져옴
# 이는 어느 셀 내에서 진행 바를 표시할 수 있게 해줌
from tqdm.notebook import tqdm

# 학습 메트릭 정의
# 평균손실을 추적하는 매트릭 객체를 생성함
train_loss = tf.keras.metrics.Mean(name='train_loss')
# 평균정확도를 추적하는 매트릭 객체를 생성함
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

# 모델 학습
# 에포크 설정 : 총 20번의 에포크로 학습을 수행함
EPOCHS = 20
# 에포크 반복 : 0부터 19까지 에포크를 반복함
for epoch in range(EPOCHS):
    start = time.time()
    # 메트릭 초기화 : 각 에포크마다 손실과 정확도 메트릭을 초기화함
    train_loss.reset_state()
    train_accuracy.reset_state()
    # 에포크 시작 메시지 출력
    print(f"에포크 {epoch + 1}/{EPOCHS} 시작...")

    # 배치 순회 및 학습 스텝 수행
    # 데이터셋을 순회하며 각 배치를 처리함, tqdm은 진행상황을 시각적으로 보여줌
    for (batch, (inp, tar)) in tqdm(enumerate(dataset), total=len(dataset)):
        # 각 배치에 대해 트레인 스텝 함수를 호출하여 모델을 학습시킴
        train_step(inp['inputs'], tar['outputs'])

    # 에포크 완료 후 결과 출력
    epoch_time = time.time() - start # 현재 에포크의 소요시간을 측정함
    # 에포크 종료 시 손실과 정확도, 그리고 소요시간을 출력함
    print(f'에포크 {epoch + 1} 완료 - 손실: {train_loss.result():.4f}, 정확도: {train_accuracy.result():.4f}, 시간: {epoch_time:.2f}초')

    # 에포크가 끝날 때마다 체크포인트 저장, 에포크가 5의 배수일 때만 체크포인트를 저장함
    if (epoch + 1) % 5 == 0:
        # 현재 모델의 상태를 체크포인트 파일로 저장함
        ckpt_save_path = ckpt_manager.save()
        print(f'에포크 {epoch + 1}에 체크포인트 저장: {ckpt_save_path}')
    # 구분선 출력
    print("-" * 50)
# 위 코드는 트랜스포머 모델을 20번 에포크동안 학습시키며, 각 에포크의 손실과 정확도를 출력하고
# 5의 배수 에포크마다 체크포인트를 저장함, 또한 tqdm을 사용하여 학습 진행 상황을 시각적으로 보여줌

에포크 1/20 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 1 완료 - 손실: 9.4968, 정확도: 0.1683, 시간: 65.54초
--------------------------------------------------
에포크 2/20 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 2 완료 - 손실: 7.7962, 정확도: 0.2127, 시간: 19.37초
--------------------------------------------------
에포크 3/20 시작...


...

에포크 18 완료 - 손실: 0.9024, 정확도: 0.8179, 시간: 19.12초
--------------------------------------------------
에포크 19/20 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 19 완료 - 손실: 0.7530, 정확도: 0.8423, 시간: 19.38초
--------------------------------------------------
에포크 20/20 시작...


  0%|          | 0/185 [00:01<?, ?it/s]

에포크 20 완료 - 손실: 0.6470, 정확도: 0.8603, 시간: 20.12초
에포크 20에 체크포인트 저장: ./checkpoints/transformer/ckpt-4
--------------------------------------------------


In [28]:
# 체크포인트 복원
# 가장 최근의 체크포인트에서 모델의 상태를 복원함. 만약 최신 체크포인트가 존재하면 해당 체크포인트에서
# 모델의 가중치와 옵티마이저 상태를 불러옴
ckpt.restore(ckpt_manager.latest_checkpoint)
# 체크포인트 확인 및 시작 에포크 설정
# 체크포인트가 존재하는지 여부를 확인함
if ckpt_manager.latest_checkpoint:
    # 체크포인트가 복원되었음을 알리는 메시지를 출력함
    print(f"체크포인트에서 복원됨: {ckpt_manager.latest_checkpoint}")
    # 체크포인트 파일이름에서 에포크번호를 추출하여 start_epoch변수에 저장함
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
# 체크포인트가 없는 경우 처음부터 학습을 시작함
else:
    print("체크포인트가 없습니다. 처음부터 학습을 시작합니다.")
    start_epoch = 0

# 이어서 학습 (start_epoch부터 100까지)
EPOCHS = 100 # 100번의 에포크
# 에포크 반복 : start_each부터 99까지의 에포크를 반복함
for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    # 메트릭 초기화 : 각 에포크마다 손실과 정확도 메트릭을 초기화함
    train_loss.reset_state()
    train_accuracy.reset_state()
    # 에포크 시작 메시지 출력
    print(f"에포크 {epoch + 1}/{EPOCHS} 시작...")
    # 배치 순회 및 학습 스텝 수행
    # 데이터셋을 순회하며 각 배치를 처리하고, tqdm을 사용하여 진행 상황을 시각적으로 보여줌
    for (batch, (inp, tar)) in tqdm(enumerate(dataset), total=len(dataset)):
        # 각 배치에 대해 트레인_스텝 함수를 호출하여 모델을 학습시킴
        train_step(inp['inputs'], tar['outputs'])
    # 에포크 종료 후 결과 출력
    # 현재 에포크의 소요 시간을 측정함
    epoch_time = time.time() - start
    # 에포크 종료 시 손실과 정확도, 그리고 소요시간을 출력함
    print(f'에포크 {epoch + 1} 완료 - 손실: {train_loss.result():.4f}, 정확도: {train_accuracy.result():.4f}, 시간: {epoch_time:.2f}초')
    # 체크포인트 저장
    # 에포크가 5의 배수일 때만 체크포인트를 저장함
    if (epoch + 1) % 5 == 0:
        # 현재 모델의 상태를 체크포인트 파일로 저장함
        ckpt_save_path = ckpt_manager.save()
        print(f'에포크 {epoch + 1}에 체크포인트 저장: {ckpt_save_path}')
    # 구분선 출력
    print("-" * 50)
# 위 코드는 체크포인트에서 모델을 복원하고 해당 지점부터 100번의 에포크까지 학습을 계속함
# 체크포인트가 없을 경우 처음부터 학습을 시작하며 학습 진행상황을 tqdm을 통해 시각적으로 보여줌

체크포인트에서 복원됨: ./checkpoints/transformer/ckpt-4
에포크 5/100 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 5 완료 - 손실: 0.5627, 정확도: 0.8745, 시간: 25.26초
에포크 5에 체크포인트 저장: ./checkpoints/transformer/ckpt-5
--------------------------------------------------
에포크 6/100 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 6 완료 - 손실: 0.5047, 정확도: 0.8833, 시간: 19.21초
--------------------------------------------------
에포크 7/100 시작...


...

에포크 98 완료 - 손실: 0.0639, 정확도: 0.9930, 시간: 20.31초
--------------------------------------------------
에포크 99/100 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 99 완료 - 손실: 0.0727, 정확도: 0.9933, 시간: 19.32초
--------------------------------------------------
에포크 100/100 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 100 완료 - 손실: 0.0532, 정확도: 0.9962, 시간: 19.36초
에포크 100에 체크포인트 저장: ./checkpoints/transformer/ckpt-24
--------------------------------------------------


In [29]:
# 모델 체크포인트 로드 (학습 이후)
# 저장된 체크포인트가 있으면 최신 체크포인트를 복원
# 체크포인트 존재 여부 확인
if ckpt_manager.latest_checkpoint:
    # 체크포인트 복원 : 최신 체크포인트에서 모델의 상태를 복원함, 이 과정에서 모델의 가중치와 옵티마이저 상태가
    # 체크포인트로부터 불러와짐
    ckpt.restore(ckpt_manager.latest_checkpoint)
    # 복원 확인 메시지 출력
    # 복원이 완료되었음을 알리는 메시지를 출력함, 메시지는 복원된 체크포인트의 경로를 포함함
    print(f'최신 체크포인트 복원: {ckpt_manager.latest_checkpoint}')
# 위 코드는 모델 학습이 끝난 후 또 다른 이유로 모델을 다시 사용할 때 유용함, 최신 체크포인트가 존재하면
# 해당 체크포인트에서 모델의 상태를 복원하여 이전 학습 상태를 유지할 수 있음. 이를 통해 학습을
# 중단했던 지점에서 바로 이어서 학습을 재개하거나 모델을 테스트 및 추론에 사용할 수 있음

최신 체크포인트 복원: ./checkpoints/transformer/ckpt-24


In [30]:
# 인코더와 디코더를 사용하여 응답을 생성하는 함수를 정의함
# 응답 생성 함수 정의
def evaluate(sentence):
    # 문장 전처리 : 입력 문장을 전처리함수(preprocess_sentence)를 사용하여 전처리함
    # 이 함수는 일반적으로 특수문자제거, 소문자변환 등의 작업을 수행함
    sentence = preprocess_sentence(sentence)

    # 문장 토큰화
    # 전처리된 문장을 토크나이저를 사용해 인코딩함, 이는 문장의 각 단어를 정수 인덱스로 변환함
    encoder_input = tokenizer.encode(sentence)
    # 인코더 입력에서 시작토큰과 종료토큰을 추가함, 이는 모델이 입력 문장의 시작과 끝을 인식하도록 도움
    encoder_input = [START_TOKEN_ID] + encoder_input + [END_TOKEN_ID]
    # 텐서의 차원을 확장하여 배치크기가 1인 텐서로 만듬
    encoder_input = tf.expand_dims(encoder_input, 0)

    # 디코더 입력 초기화
    decoder_input = [START_TOKEN_ID] # 디코더의 입력을 시작 토큰으로 초기화함
    output = tf.expand_dims(decoder_input, 0) # 디코더 입력을 배치 차원이 1인 텐서로 확장함

    # 자동 회귀(auto-regressive) 방식으로 출력 생성
    # 반복문을 통한 예측 : 아래 루프를 사용하여 최대 길이만큼 반복
    for i in range(MAX_LENGTH):
        # 트랜스포머 모델에 인코더 입력과 디코더 입력을 전달하여 예측값을 얻음, training=False는 모델을
        # 평가모드로 설정함
        predictions = transformer(
            {
                'inputs': encoder_input,
                'dec_inputs': output
            },
            training=False
        )
        # 마지막 토큰의 예측에서 다음 토큰 선택
        # 예측값에서 마지막 시퀀스의 마지막 토큰에 해당하는 부분을 추출함
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        # 가장 높은 확률을 가진 토큰의 인덱스를 선택하고 이를 정수형으로 변환함
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)  # int32로 명시적 변환

        # 종료 토큰이 예측되면 반환(종료조건 : 예측된 토큰이 종료 토크이면 반복을 종료함)
        if tf.equal(predicted_id, END_TOKEN_ID):  # tf.equal 사용
            break

        # 예측된 ID를 디코더 입력에 연결
        # 예측된 토큰을 디코더 입력에 추가하여 다음 반복에 사용함
        output = tf.concat([output, predicted_id], axis=-1)

    # 출력 텍스트로 변환 : 디코더의 출력텐서를 토크나이저를 사용하여 원래 텍스트로 인코딩함, 이때 출력에서
    # 토크나이저의 vocab_size보다 작은 인덱스만 선택하여 유효한 텍스트만 포함되도록 함
    output_text = tokenizer.decode([i for i in output[0].numpy() if i < tokenizer.vocab_size])
    # 결과 반환 : 생성된 응답 텍스트를 반환함
    return output_text
# 위 함수는 주어진 문장에 대해 트랜스포머 모델을 사용하여 응답을 생성함, 자동 회귀 방식으로 한 단어씩 예측하며,
# 종료 토큰이 예측되거나 최대 길이에 도달하면 생성을 멈춤, 최종적으로 생성된 응답을 텍스트 형식으로 변환함

In [31]:
# 챗봇 응답 함수 : 주어진 문장에 대해 챗봇의 응답을 생성하고 생성된 응답에서 불필요한 특수 토큰을 제거하여
# 정리하는 함수를 정의함.
# 챗봇 응답 함수 정의
def chat_response(sentence):
    # 응답생성 : evaluate함수를 호출하여 입력 문장에 대한 응답을 생성함, 이 함수는 앞서 정의된 트랜스포머 모델을
    # 사용하여 문장에 대한 응답 텍스트를 생성함
    result = evaluate(sentence)

    # 특수 토큰 제거 및 정리: result.replace() : 응답 텍스트에서 빈 문자열을 제거함, 이는 토큰화 과정에서 추가된
    # 불필요한 빈 문자열을 제거하기 위함. .replace() : 추가로 인코더와 디코더 입력에 사용된 특수 토큰을 제거함
    # .strip() : 문자열 앞뒤의 공백을 제거하여 응답을 깔끔하게 정리함
    result = result.replace('', '').replace('', '').strip()
    # 정리된 응답 텍스트를 반환함
    return result
# 위 함수는 evaluate함수를 통해 생성된 응답을 받아 특수 토큰과 불필요한 공백을 제거하여 최종적인 챗봇의 응답을 제공
# 이를 통해 사용자에게 보다 자연스러운 응답을 보여줄 수 있음

In [32]:
# 몇 가지 질문에 대한 응답 테스트
# 질문목록 정의, 테스트할 질문 형태를 리스트 형태로 정의함
# 각 질문은 사용자가 챗봇에게 할 수 있는 예시들로 구성됨
test_questions = [
    '안녕하세요?',
    '오늘 날씨가 어때요?',
    '당신의 이름은 뭐예요?',
    '취미가 뭐예요?',
    '한국어 잘하시네요'
]
# 리스트에 있는 각 질문을 하나씩 반복함
for question in test_questions:
    # 현재 질문 내용을 출력함
    print('질문:', question)
    # 현재 질문에 대한 챗봇의 응답을 생성하고 그 결과를 출력함, 각 질문과 응답 쌍 사이에
    # 빈 줄을 추가하여 출력을 깔끔하게 정리함
    print('응답:', chat_response(question))
    print()
# 위 코드는 정의된 질문들에 대해 챗봇의 응답을 테스트하고 각 질문에 대한 응답을 콘솔에 출력함
# 이를 통해 챗봇이 다양한 질문에 어떻게 반응하는지를 확인할 수 있으며 모델의 성능을 간단히 평가할 수 있음
# 챗_리스판스 함수는 앞서 정의된 evaluate함수를 사용하여 응답을 생성하고 불필요한 토큰을 제거하여
# 정리된 형태의 응답을 제공함

질문: 안녕하세요?
응답: 마음이 가장 싫을 거예요 .

질문: 오늘 날씨가 어때요?
응답: 마음이 정리되길 바랄게요 .

질문: 당신의 이름은 뭐예요?
응답: 마음이 가장 확실하겠죠 .

질문: 취미가 뭐예요?
응답: 큰 문제네요 .

질문: 한국어 잘하시네요
응답: 무엇이든 물어보세요 .



In [33]:
# 대화형 챗봇 인터페이스
# 대화형 챗봇 인터페이스 정의
def interactive_chat():
    # 대화시작 안내 : 사용자에게 챗봇과의 대화가 시작됨을 알리고 종료라는 단어를 입력하면
    # 대화가 종료된다는 안내메시지를 출력함
    print("한국어 챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 종료됩니다.")
    # 화면을 구분하기 위해 긴 구분선을 출력함
    print('=' * 80)
    # 무한 루프를 통한 대화 진행
    while True: # 무한 루프를 시작하여 사용자가 '종료'를 입력할 때까지 계속해서 대화를 진행
        # 사용자로부터 입력을 받음, 입력 프롬프트는 '사용자>'로 표시됨
        user_input = input('사용자 > ')
        # 종료 조건 확인
        # 사용자 입력이 '종료'인지 확인, 대소문자를 구분하지 않기 위해 lower()메소드를 사용함
        if user_input.lower() == '종료':
            # 종료조건을 만족하면 대화 종료를 알리는 메시지를 출력함
            print('챗봇과의 대화를 종료합니다.')
            # 무한 루프를 빠져나감
            break
        # 응답 생성 및 출력
        response = chat_response(user_input) # 사용자의 입력에 대해 chat_response함수를 호출하여 챗봇의 응답을 생성함
        # 생성된 응답을 출력함, 출력 형식은 '챗봇>'로 시작하여 사용자가 입력한 내용과 챗봇의 응답을 구분함
        print('챗봇 >', response)
        # 응답 출력 후 빈 줄을 추가하여 다음 입력을 위한 공간을 확보함
        print()
# 위 함수는 사용자가 직접 입력한 문장에 대해 실시간으로 챗봇의 응답을 제공하며 '종료'라는 단어를 입력하면
# 대화가 종료됨, 이를 통해 사용자는 챗봇과 자연스럽게 대화할 수 있는 환경을 경험할 수 있음
# chat_response함수는 앞서 정의한 대로 입력 문장을 전처리하고 모델의 예측을 통해 응답을 생성한 후
# 불필요한 토큰을 제거하여 정리된 형태의 응답을 제공함

In [34]:
# 대화형 인터페이스 실행
interactive_chat()

한국어 챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 종료됩니다.
사용자 > 안녕
챗봇 > 더 올 거예요 .

사용자 > 그만 할게.
챗봇 > 마음이 가장 확실하겠죠 .

사용자 > 종료
챗봇과의 대화를 종료합니다.


In [35]:
# 대화형 인터페이스 실행
interactive_chat()

한국어 챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 종료됩니다.
사용자 > 반갑습니다.
챗봇 > 매력이겠죠 .

사용자 > 테스트입니다.
챗봇 > 마음이 가장 확실하겠죠 .

사용자 > 종료
챗봇과의 대화를 종료합니다.


In [36]:
# 디버깅용 함수 : 디버깅 목적으로 응답을 생성하는 함수 정의, 테스트
# 디버깅 함수 정의
def debug_evaluate(sentence):
    # 문장 전처리 : 입력 문장을 전처리함수를 사용하여 전처리함, 이 단계에서는
    # 주로 문장에서 불필요한 문자를 제거하거나 소문자로 변환하는 등의 작업이 이루어짐
    sentence = preprocess_sentence(sentence)

    # 전처리된 문장을 출력함
    print("전처리된 문장:", sentence)

    # 문장 토큰화 : 전처리된 문장을 토크나이저를 사용하여 인코딩함, 이는 문장의 각 단어를
    # 정수 인덱스로 변환함
    encoder_input = tokenizer.encode(sentence)
    # 인코딩된 입력 토큰을 출력함
    print("인코더 입력 토큰:", encoder_input)
    # 인코딩된 토큰을 다시 디코딩하여 원래 텍스트로 변환된 모습을 확인함
    print("인코더 입력 토큰 디코딩:", tokenizer.decode(encoder_input))
    # 인코더 입력 준비
    # 인코더 입력에 시작 토큰과 종료 토큰을 추가함
    encoder_input = [START_TOKEN_ID] + encoder_input + [END_TOKEN_ID]
    # 텐서 차원을 확장하여 배치 크기가 1인 텐서로 만듬
    encoder_input = tf.expand_dims(encoder_input, 0)

    # 디코더 입력 초기화
    # 디코더 입력을 시작 토큰으로 초기화함
    decoder_input = [START_TOKEN_ID]
    # 디코더 입력을 배치 차원이 1인 텐서로 확장함
    output = tf.expand_dims(decoder_input, 0)

    # 각 스텝의 출력 저장 : 각 스텝에서 생성된 토큰을 저장할 리스트를 초기화함
    step_outputs = []

    # 자동 회귀(auto-regressive) 방식으로 출력 생성
    # 반복문을 통한 예측 : 루프를 사용하여 최대 길이만큼 반복함
    for i in range(MAX_LENGTH):
        # 트랜스포머 모델에 인코더 입력과 디코더 입력을 전달하여 예측값을 얻음
        predictions = transformer(
            {
                'inputs': encoder_input,
                'dec_inputs': output
            },
            # 모델을 평가모드로 설정함
            training=False
        )

        # 마지막 토큰의 예측에서 다음 토큰 선택
        # 토큰 선택 : predictions[]을 통해 마지막 시퀀스의 마지막 토큰에 해당하는
        # 부분을 추출하고 tf.argmax를 사용하여 가장 높은 확률을 가진 토큰의 인덱스를 선택함
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        # 토큰 디코딩 및 저장 : 선택된 토큰의 id와 텍스트를 확인하고 step_outputs리스트에 저장함
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 생성된 토큰 확인
        token_id = predicted_id.numpy()[0][0]
        token_text = tokenizer.decode([token_id]) if token_id < tokenizer.vocab_size else f""
        step_outputs.append((token_id, token_text))

        # 예측된 토큰이 종료 토큰이면 반복을 종료함
        if tf.equal(predicted_id, END_TOKEN_ID):
            break

        # 예측된 토큰을 디코더 입력에 추가하여 다음 반복에 사용함
        output = tf.concat([output, predicted_id], axis=-1)

    # 각 스텝 출력 확인
    print("\n생성된 토큰 시퀀스 (단계별):")
    # 각 스텝에서 생성된 토큰을 출력함, 각 스텝의 인덱스와 토큰 id 그리고 텍스트를 함께 보여줌
    for i, (token_id, token_text) in enumerate(step_outputs):
        print(f"스텝 {i+1}: ID={token_id}, 텍스트='{token_text}'")

    # 전체 출력 토큰 확인 : 최종적으로 생성된 모든 토큰을 배열 형태로 출력함
    full_output = output[0].numpy()
    # 전체 토큰 배열을 출력함
    print("\n전체 출력 토큰:", full_output)

    # 출력 텍스트로 변환 : 디코더의 출력 텐서를 토크나이저를 사용하여 원래 텍스트로 디코딩함.
    # 이때 출력에서 토크나이저의 vocab_size보다 작은 인덱스만 선택하여 유효한 텍스트만 포함되도록 함
    output_text = tokenizer.decode([i for i in full_output if i < tokenizer.vocab_size])
    # 최종적으로 생성된 응답 텍스트를 출력함
    print("최종 출력 텍스트:", output_text)
    # 생성된 응답 텍스트를 반환함
    return output_text

# 테스트
test_sentence = "안녕하세요?" # 테스트할 문장을 정의함
debug_result = debug_evaluate(test_sentence) # debug_evaluate 함수를 호출하여 테스트 문장에 대한 디버깅 정보를 출력하고, 최종 응답을 얻음

# 위 함수는 주어진 문장에 대해 트랜스포머 모델을 사용하여 응답을 생성하는 과정을 단계별로 디버깅할 수 있도록 도와줌, 각 단계에서 토큰과 최종응답을
# 출력하여, 모델의 동작을 이해하고 문제를 진단하는데 유용함

전처리된 문장: 안녕하세요 ?
인코더 입력 토큰: [2183, 2]
인코더 입력 토큰 디코딩: 안녕하세요 ?

생성된 토큰 시퀀스 (단계별):
스텝 1: ID=31, 텍스트='마음이 '
스텝 2: ID=238, 텍스트='가장 '
스텝 3: ID=15214, 텍스트='싫을 '
스텝 4: ID=3, 텍스트='거예요'
스텝 5: ID=1, 텍스트=' .'
스텝 6: ID=21835, 텍스트=''

전체 출력 토큰: [21834    31   238 15214     3     1]
최종 출력 텍스트: 마음이 가장 싫을 거예요 .


In [37]:
# 토크나이저 저장
# 피클 모듈 임포트하기, 이 모듈은 파이선 객체를 직렬화하여 파일에 저장하거나 네트워크를 통해 전송할 수 있게 해줌
import pickle
# 'tokenizer.pickle'라는 이름의 파일을 바이너리 쓰기 모드(wb)로 염, with문을 사용하여 파일을 자동으로 닫음
with open('tokenizer.pickle', 'wb') as handle:
    # 객체 직렬화 및 저장
    # 현재 토크나이저, 시작토큰id, 종료토큰id, 그리고 어휘크기를 포함하는 딕셔너리를 파일에 저장함
    pickle.dump({
        'tokenizer': tokenizer,
        'start_token': START_TOKEN_ID,
        'end_token': END_TOKEN_ID,
        'vocab_size': VOCAB_SIZE
    # handle 매개변수는 저장할 파일 핸들을 나타내며 protocol은 최신 직렬화 포맷을 사용하여 저장함
    }, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 저장완료 메세지 출력
print('토크나이저 저장 완료')

# 위 코드는 토크나이저와 관련된 중요한 정보를 파일로 저장하여 이후의 모델을 재사용하거나 다른 환경에서
# 동일한 토크나이저를 사용할 수 있도록 함, pickle모듈을 사용하면 파이선 객체를 그대로 파일에 저장할 수 있어,
# 복잡한 구조의 객체도 쉽게 직렬화할 수 있음, 이렇게 저장된 토크나이저는 필요할 때 다시 불러와서 사용할 수 있음

토크나이저 저장 완료


In [38]:
# 토크나이저 불러오기 함수 및 테스트
def load_tokenizer():
    # 파일 읽기 : 'tokenizer.pickle'파일을 바이너리 읽기 모드(rb)로 염, with문을 사용하여 파일을
    # 자동으로 닫음
    with open('tokenizer.pickle', 'rb') as handle:
        # 파일에서 데이터를 읽어와 역직렬화함, 이 과정에서 저장된 토크나이저와 관련한 정보들이
        # 다시 객체로 복원됨
        data = pickle.load(handle)
    # 정보반환 : 역직렬화된 데이터에서 토크나이저, 시작토큰id, 종료토큰id 그리고 어휘크기를 반환함
    return data['tokenizer'], data['start_token'], data['end_token'], data['vocab_size']

# 테스트
# 토크나이저 불러오기, 함수를 호출하여 저장된 토크나이저와 관련된 정보를 불러옴
loaded_tokenizer, loaded_start, loaded_end, loaded_vocab_size = load_tokenizer()
# 정보출력 : 불러온 어휘사전의 크기를 출력하여 확인함
print(f'불러온 어휘 사전 크기: {loaded_vocab_size}')

# 위 코드는 저장된 토크나이저와 관련된 정보를 파일에서 불러와 사용할 수 있도록 함, pickle모듈을
# 사용하여 직렬화된 데이터를 역직렬화함으로써 이전에 저장한 객체들을 그대로 복원할 수 있음. 이를 통해
# 모델을 다른 환경에서 재사용하거나, 토크나이저를 다시 설정할 필요없이 불러온 정보를 사용할 수 있음
# 테스트 부분에서는 불러온 어휘 사전의 크기를 출력하여 정상적으로 불러왔는지 확인

불러온 어휘 사전 크기: 21836
